# MORFEUSZ & CONCRAFT - PREANNOTATION

### IMPORTS, VARIABLES

In [1]:
# https://github.com/kawu/concraft-pl
# https://github.com/kawu/concraft-pl/tree/master/bindings/python

In [2]:
import sys
import pandas as pd
import os
sys.path.append('../concraft-pl/bindings/python/')

In [82]:
from morfeusz2 import Morfeusz
from concraft_pl2 import Concraft, Server

server = Server(model_path='../concraft-pl/model-SGJP.gz')
file = '../data/memoirs_annotated_10k.txt'
original_file = '../data/memoirs_10k.txt'

In [77]:
morfeusz = Morfeusz(expand_tags=True)
concraft = Concraft()

### FUNCTIONS AND CLASSES

In [5]:
def best_interpretation(dag_disamb: list):
    '''A function that allows for the selection of only the best possible morphosyntactic interpretation of a sentence as
    returned by Morfeusz2 + Concraft.
    
    Args:
        dag_disamb (list): A list of possible interpretations returned by Concraft based on Morfeusz2's analysis.
        
    Returns:
        A list containing only the highest probability interpretations for every token.    
    '''
    
    while("" in dag_disamb):
        dag_disamb.remove("")
    
    best_inter = []
    
    for item in dag_disamb:
        if item[0] == len(best_inter):
            best_inter.append(item)
        else:
            if item[3] > best_inter[-1][3]:
                best_inter[-1] = item
        
            
    return best_inter

In [51]:
class OriginalAnnotations:
    '''A class intended to process and store the tokens and their respective annotations from the original text.
    
    Attributes:
        tokens (list[str]): A list of all the tokens (without annotation) in the original data. Every element of the list is a string.
        gold_standard (list[str]): A list of all the original annotations (without tokens). Every element is a string.
        sentences (list[str]): A list of the original sentences (either truly original, if available, or reconstructed).
        sentences_tokenized (list[list[str]]): A list of lists representing tokenized, unannotated sentences.
        gold_standard_tokenized (list[list[str]]): A list of lists representing the gold standard annotations per sentence.
        
    '''
    def __init__(self, filename: str, lowercase: bool = False, original: str = '', nested: bool = True):
        '''The __init__ method of the class.
        Constructs the token and gold_standard lists.
        
        Args:
            filename (str): The name of the file that the text and the annotations are to be obtained from.
            lowercase (bool): Determines whether the tokens should be lowercased or if original capitalization should be retained.
            original (str): The name of the file containing the original unannotated sentences. If left empty or invalid,
                reconstructed sentences will be used.
        '''
        # opening the annotated text
        text = []
        with open(filename) as f:
            lines = f.readlines()
            for line in lines:
                text.append(line.strip())     
        
        # setting up attributes
        self.tokens = []
        self.gold_standard = []
        
        self.sentences = []
        
        self.sentences_tokenized = []
        self.gold_standard_tokenized = []
        
        # reading in unannotated sentences, if applicable
        if os.path.exists(original):
            with open(original) as f:
                lines = f.readlines()
                for line in lines:
                    self.sentences.append(line.strip())
        else:
            original = ''
        
        # splitting tokens and annotation
        for sentence in text:
            # setting up temporary lists to store the values per sentence
            temp_sent = []
            temp_anns = []
            temp_conc = []
            temp_conc_ann = []
            # setting up a string for potential concatenation of split elements
            full_sent = ''
            # setting up variables controlling the extent of concatenation / grouping together of elements
            concatenate = False
            concatenate_next = False
            
            sentence = sentence.strip()
            sentence = sentence.split(" ")
            
            for annotated_token in sentence:
                
                if concatenate_next == True:
                    concatenate = False
                    concatenate_next = False
                
                split_token = annotated_token.split('_')
                
                if split_token[0][0] == '[':
                    split_token[0] = split_token[0][1:]
                    concatenate = True
                    
                if split_token[-1][-1] == ']':
                    split_token[-1] = split_token[-1][:-1]
                    concatenate_next = True
                    
                if concatenate:
                    full_sent += split_token[0]
                    temp_conc.append(split_token[0])
                    temp_conc_ann.append(split_token[1])
                else:
                    full_sent += (split_token[0] + ' ')
                    if len(temp_conc) != 0:
                        temp_sent.append(temp_conc)
                        temp_anns.append(temp_conc_ann)
                        temp_conc = []
                        temp_conc_ann = []
                
                if not lowercase:
                    self.tokens.append(split_token[0])
                    if not concatenate:
                        temp_sent.append(split_token[0])
                else:
                    self.tokens.append(split_token[0].lower())
                    if not concatenate:
                        temp_sent.append(split_token[0].lower())
                self.gold_standard.append(split_token[1])
                temp_anns.append(split_token[1])
            
            self.sentences_tokenized.append(temp_sent)
            self.gold_standard_tokenized.append(temp_anns)
            
            # reconstructing sentences if necessary
            if original == '':
                # removing unnecessary whitespace
                full_sent = full_sent.replace(' .', '.')
                full_sent = full_sent.replace(' ,', ',')
                full_sent = full_sent.replace(' !', '!')
                full_sent = full_sent.replace(' ?', '?')
                full_sent = full_sent.replace(' :', ':')
                full_sent = full_sent.replace('„ ', '„')
                full_sent = full_sent.replace(' ”', '”')
                full_sent = full_sent.replace('( ', '(')
                full_sent = full_sent.replace(' )', ')')
                full_sent = full_sent.strip()
                self.sentences.append(full_sent)
                
        # removing the "nested" lists from tokenized sentences and their annotations that represent
        # elements that should go together
        if not nested:
            for i, sent in enumerate(self.sentences_tokenized):
                temp_sent = []
                for token in sent:
                    if isinstance(token, str):
                        temp_sent.append(token)
                    else:
                        for element in token:
                            temp_sent.append(element)
                self.sentences_tokenized[i] = temp_sent
                
            for i, sent in enumerate(self.gold_standard_tokenized):
                temp_ann = []
                for token in sent:
                    if isinstance(token, str):
                        temp_ann.append(token)
                    else:
                        for element in token:
                            temp_ann.append(element)
                self.gold_standard_tokenized[i] = temp_ann
                
    def __len__(self):
        '''The __len__ magic method of the class.
            
        Returns:
            The length of self.tokens, which should be identical to the length of self.gold_standard.
        '''
        return len(self.tokens)
        
    def __getitem__(self, index: int):
        '''The __getitem__ magic method of the class.
            
        Args:
            index (int): The index signifying the desired element.
            
        Returns:
            A string representing the combination of the original token and annotation.
        '''
        token = self.tokens[index]
        annotation = self.gold_standard[index]
        item = '_'.join([token, annotation])
            
        return item
    
    def frequencies(self):
        '''A method of the class indended for displaying raw and relative frequencies of word classes in the annotation.
        
        Returns:
            A dataframe representing the POS tag, raw frequency, relative frequency.
        '''
        freqs = []
        for item in list(set(self.gold_standard)):
            raw = self.gold_standard.count(item)
            relative = raw / len(self.gold_standard)
            
            freqs.append([item, raw, relative])
            
        freq_pd = pd.DataFrame(freqs, columns=['POS', 'raw', 'relative']).sort_values('relative', ascending=False).set_index('POS')
            
        return freq_pd

In [52]:
# conllu format
# blank lines for sent boundaries, comment lines with hash
# ID - word index, can be a range for multiword tokens, starts at 1
# FORM - word form
# LEMMA - word lemma
# UPOS - UPOS tag
# XPOS - XPOS tag
# FEATS - morphological features
# HEAD 
# DEPREL
# DEPS
# MISC
# use _ if empty
# fields are tab separated

In [97]:
class ConlluWord():
    def __init__(
        self, ID: str, FORM: str, LEMMA: str, UPOS: str, XPOS: str, FEATS: str = '_', 
        HEAD: str = '_', DEPREL: str = '_', DEPS: str = '_', MISC: str = '_'
    ):
        self.ID = ID
        self.FORM = FORM
        self.LEMMA = LEMMA
        self.UPOS = UPOS
        self.XPOS = XPOS
        self.FEATS = FEATS
        self.HEAD = HEAD
        self.DEPREL = DEPREL
        self.DEPS = DEPS
        self.MISC = MISC
        
    def return_line(self):
        elements = [
            self.ID, self.FORM, self.LEMMA, self.UPOS, self.XPOS, self.FEATS, self.HEAD, 
            self.DEPREL, self.DEPS, self.MISC]
        line = "\t".join(elements)
        return line

In [105]:
class ConlluFormatter():
    def __init__(
        self, 
        sents: list, 
        tokenized: list, 
        anns: list, 
        morfeusz=morfeusz, 
        concraft=concraft
    ):
        # sanity check
        if len(sents)!=len(tokenized) or len(sents)!=len(anns):
            print('Input lists not of equal length.')
            return
        
        for i, sent in enumerate(sents):
            dag = morfeusz.analyse(sent)
            dag_disamb = concraft.disamb(dag)
            best_inter = best_interpretation(dag_disamb)
            
            for j, inter in enumerate(best_inter):
                idx = str(j + 1)
                form = inter[2][0]
                lemma = inter[2][1].split(':')[0]
                upos = '_'
                xpos = inter[2][2]
                
                word = ConlluWord(idx, form, lemma, upos, xpos)
                print(word.return_line())
            
            
            
            if len(best_inter) != len(tokenized[i]):
                print(i)
                print(len(best_inter))
                print(len(tokenized[i]))
            
            if i == 0:
                for j, token in enumerate(tokenized[0]):
                    print(best_inter[j])
                    print(token)
                    print()
        
        

### TESTING AROUND

In [106]:
word = ConlluWord('id', 'form', 'lemma', 'upos', 'xpos')
print(word.return_line())

id	form	lemma	upos	xpos	_	_	_	_	_


In [107]:
formatter = ConlluFormatter(original_text.sentences, original_text.sentences_tokenized, original_text.gold_standard_tokenized)

1	Dziad	Dziad	_	subst:sg:acc:f	_	_	_	_	_
2	mój	mój	_	adj:sg:nom:m3:pos	_	_	_	_	_
3	Melchior	melchior	_	subst:sg:nom:m3	_	_	_	_	_
4	urodzony	urodzony	_	adj:sg:nom:m3:pos	_	_	_	_	_
5	roku	rok	_	subst:sg:gen:m3	_	_	_	_	_
6	1741	1741	_	dig	_	_	_	_	_
7	we	w	_	prep:loc:wok	_	_	_	_	_
8	wsi	wieś	_	subst:sg:loc:f	_	_	_	_	_
9	Godziszewo	Godziszewo	_	subst:sg:nom:n:ncol	_	_	_	_	_
10	parafii	parafia	_	subst:sg:gen:f	_	_	_	_	_
11	Rypnin	Rypnin	_	subst:sg:nom:m1	_	_	_	_	_
12	syn	syn	_	subst:sg:nom:m1	_	_	_	_	_
13	komornika	komornik	_	subst:sg:gen:m1	_	_	_	_	_
14	ziemskiego	ziemski	_	adj:sg:gen:m1:pos	_	_	_	_	_
15	Jana	Jan	_	subst:sg:gen:m1	_	_	_	_	_
16	Komornika	Komornik	_	subst:sg:acc:m1	_	_	_	_	_
17	ziemi	ziemia	_	subst:sg:gen:f	_	_	_	_	_
18	Dobrrzyńskiej	Dobrrzyńskiej	_	adj:sg:gen:f:pos	_	_	_	_	_
19	(	(	_	interp	_	_	_	_	_
20	a	a	_	conj	_	_	_	_	_
21	nie	nie	_	part	_	_	_	_	_
22	Antoniego	Antoni	_	subst:sg:acc:m1	_	_	_	_	_
23	–	–	_	interp	_	_	_	_	_
24	Antoni	Antoni	_	subst:sg:nom:m1	_	_	_	_	_
25	był

1	Przyjechawszy	przyjechać	_	pant:perf	_	_	_	_	_
2	do	do	_	prep:gen	_	_	_	_	_
3	Skolego	Skole	_	subst:sg:gen:n:ncol	_	_	_	_	_
4	przekonał	przekonać	_	praet:sg:m1:perf	_	_	_	_	_
5	się	się	_	part	_	_	_	_	_
6	,	,	_	interp	_	_	_	_	_
7	że	że	_	comp	_	_	_	_	_
8	te	ten	_	adj:pl:nom:m3:pos	_	_	_	_	_
9	rozległe	rozległy	_	adj:pl:nom:m3:pos	_	_	_	_	_
10	dobra	dobry	_	adj:sg:nom:f:pos	_	_	_	_	_
11	w	w	_	prep:loc:nwok	_	_	_	_	_
12	górach	góra	_	subst:pl:loc:f	_	_	_	_	_
13	i	i	_	conj	_	_	_	_	_
14	lasach	las	_	subst:pl:loc:m3	_	_	_	_	_
15	ogromnych	ogromny	_	adj:pl:loc:m3:pos	_	_	_	_	_
16	żadnej	żaden	_	adj:sg:gen:f:pos	_	_	_	_	_
17	nie	nie	_	part	_	_	_	_	_
18	czynią	czynić	_	fin:pl:ter:imperf	_	_	_	_	_
19	intraty	intrata	_	subst:sg:gen:f	_	_	_	_	_
20	–	–	_	interp	_	_	_	_	_
21	i	i	_	conj	_	_	_	_	_
22	utyskiwał	utyskiwać	_	praet:sg:m1:imperf	_	_	_	_	_
23	na	na	_	prep:acc	_	_	_	_	_
24	to	to	_	subst:sg:acc:n:ncol	_	_	_	_	_
25	przed	przed	_	prep:inst:nwok	_	_	_	_	_
26	Melchiorem	melchior	_	subst:sg:inst

1	W	w	_	prep:loc:nwok	_	_	_	_	_
2	1784	1784	_	dig	_	_	_	_	_
3	roku	rok	_	subst:sg:loc:m3	_	_	_	_	_
4	w	w	_	prep:loc:nwok	_	_	_	_	_
5	kościele	kościół	_	subst:sg:loc:m3	_	_	_	_	_
6	Chyrowskim	Chyrowskim	_	adj:sg:loc:m3:pos	_	_	_	_	_
7	pobłogosławił	pobłogosławić	_	praet:sg:m3:perf	_	_	_	_	_
8	związek	związek	_	subst:sg:acc:m3	_	_	_	_	_
9	ówczesny	ówczesny	_	adj:sg:nom:m1:pos	_	_	_	_	_
10	Dziekan	Dziekan	_	subst:sg:nom:m1	_	_	_	_	_
11	i	i	_	conj	_	_	_	_	_
12	proboszcz	proboszcz	_	subst:sg:nom:m1	_	_	_	_	_
13	ksiądz	ksiądz	_	subst:sg:nom:m1	_	_	_	_	_
14	Michał	Michał	_	subst:sg:nom:m1	_	_	_	_	_
15	Józef	Józef	_	subst:sg:nom:m1	_	_	_	_	_
16	Rzepecki	Rzepecki	_	subst:sg:nom:m1	_	_	_	_	_
17	.	.	_	interp	_	_	_	_	_
1	Z	z	_	prep:gen:nwok	_	_	_	_	_
2	tego	ten	_	adj:sg:gen:n:pos	_	_	_	_	_
3	małżeństwa	małżeństwo	_	subst:sg:gen:n:ncol	_	_	_	_	_
4	były	być	_	praet:pl:f:imperf	_	_	_	_	_
5	dwie	dwa	_	num:pl:nom:f:congr:ncol	_	_	_	_	_
6	córki	córka	_	subst:pl:nom:f	_	_	_	_	_
7	i	i	_	conj	_	_	_	_	_
8	m

1	Opowiadano	opowiadać	_	imps:imperf	_	_	_	_	_
2	mi	mi	_	subst:sg:acc:n:ncol	_	_	_	_	_
3	też	też	_	part	_	_	_	_	_
4	zabawny	zabawny	_	adj:sg:acc:m3:pos	_	_	_	_	_
5	epizod	epizod	_	subst:sg:acc:m3	_	_	_	_	_
6	z	z	_	prep:gen:nwok	_	_	_	_	_
7	pożycia	pożyć	_	ger:sg:gen:n:perf:aff	_	_	_	_	_
8	moich	mój	_	adj:pl:gen:n:pos	_	_	_	_	_
9	wujostwa	wujostwo	_	subst:sg:gen:n:ncol	_	_	_	_	_
10	.	.	_	interp	_	_	_	_	_
1	Raz	raz	_	subst:sg:acc:m3	_	_	_	_	_
2	wyjechała	wyjechać	_	praet:sg:f:perf	_	_	_	_	_
3	ciotka	ciotka	_	subst:sg:nom:f	_	_	_	_	_
4	moja	mój	_	adj:sg:nom:f:pos	_	_	_	_	_
5	z	z	_	prep:gen:nwok	_	_	_	_	_
6	domu	dom	_	subst:sg:gen:m3	_	_	_	_	_
7	–	–	_	interp	_	_	_	_	_
8	w	w	_	prep:acc:nwok	_	_	_	_	_
9	jakiś	jakiś	_	adj:sg:acc:m3:pos	_	_	_	_	_
10	czas	czas	_	subst:sg:acc:m3	_	_	_	_	_
11	powraca	powracać	_	fin:sg:ter:imperf	_	_	_	_	_
12	-	-	_	interp	_	_	_	_	_
13	zastaje	zastawać	_	fin:sg:ter:imperf	_	_	_	_	_
14	katafalk	katafalk	_	subst:sg:acc:m3	_	_	_	_	_
15	oświecony	oświecony	_	subst:sg:n

1	Syn	syn	_	subst:sg:nom:m1	_	_	_	_	_
2	ożeniwszy	ożenić	_	pant:perf	_	_	_	_	_
3	się	się	_	part	_	_	_	_	_
4	z	z	_	prep:inst:nwok	_	_	_	_	_
5	cioteczną	cioteczny	_	adj:sg:inst:f:pos	_	_	_	_	_
6	siostrą	siostra	_	subst:sg:inst:f	_	_	_	_	_
7	Treterówną	Treterówną	_	subst:sg:inst:f	_	_	_	_	_
8	–	–	_	interp	_	_	_	_	_
9	założył	założyć	_	praet:sg:m1:perf	_	_	_	_	_
10	konwikt	konwikt	_	subst:sg:acc:m3	_	_	_	_	_
11	w	w	_	prep:loc:nwok	_	_	_	_	_
12	Rosyi	Rosyi	_	subst:sg:loc:n:ncol	_	_	_	_	_
13	–	–	_	interp	_	_	_	_	_
14	czy	czy	_	conj	_	_	_	_	_
15	na	na	_	prep:loc	_	_	_	_	_
16	Bukowinie	Bukowina	_	subst:sg:loc:f	_	_	_	_	_
17	–	–	_	interp	_	_	_	_	_
18	drugi	drugi	_	adj:sg:nom:m1:pos	_	_	_	_	_
19	jakiś	jakiś	_	adj:sg:nom:m1:pos	_	_	_	_	_
20	dziwak	dziwak	_	subst:sg:nom:m1	_	_	_	_	_
21	napastował	napastować	_	praet:sg:m1:perf	_	_	_	_	_
22	ludzi	człowiek	_	subst:pl:acc:m1	_	_	_	_	_
23	,	,	_	interp	_	_	_	_	_
24	aż	aż	_	comp	_	_	_	_	_
25	się	się	_	part	_	_	_	_	_
26	nim	on	_	ppron3:sg:inst:m1:ter:akc:

...

1	Pana	Pan	_	subst:sg:acc:m1	_	_	_	_	_
2	Kamińskiego	Kamiński	_	subst:sg:gen:m1	_	_	_	_	_
3	wysłano	wysłać	_	imps:perf	_	_	_	_	_
4	z	z	_	prep:inst:nwok	_	_	_	_	_
5	Marcelim	Marceli	_	subst:sg:inst:m1	_	_	_	_	_
6	mną	ja	_	ppron12:sg:inst:m1:pri	_	_	_	_	_
7	i	i	_	conj	_	_	_	_	_
8	Stanisławem	Stanisław	_	subst:sg:inst:m1	_	_	_	_	_
9	do	do	_	prep:gen	_	_	_	_	_
10	Stanisławowa	Stanisławów	_	subst:sg:gen:m3	_	_	_	_	_
11	–	–	_	interp	_	_	_	_	_
12	Marceli	Marceli	_	subst:sg:nom:m1	_	_	_	_	_
13	poszedł	pójść	_	praet:sg:m1:perf	_	_	_	_	_
14	do	do	_	prep:gen	_	_	_	_	_
15	gimnazyum	gimnazyum	_	subst:sg:gen:n:ncol	_	_	_	_	_
16	a	a	_	conj	_	_	_	_	_
17	ja	ja	_	ppron12:sg:nom:f:pri	_	_	_	_	_
18	do	do	_	prep:gen	_	_	_	_	_
19	1	1	_	dig	_	_	_	_	_
20	normalney	normalney	_	subst:sg:nom:n:ncol	_	_	_	_	_
21	–	–	_	interp	_	_	_	_	_
22	Moim	mój	_	adj:sg:inst:n:pos	_	_	_	_	_
23	Dyrektorem	dyrektor	_	subst:sg:inst:m1	_	_	_	_	_
24	był	być	_	praet:sg:m1:imperf	_	_	_	_	_
25	Bazyli	Bazyli	_	subst:sg:nom:m1	_	_	_	_	_


1	Żeby	żeby	_	comp	_	_	_	_	_
2	konie	koń	_	subst:pl:nom:m2	_	_	_	_	_
3	raźno	raźno	_	adv:pos	_	_	_	_	_
4	szły	iść	_	praet:pl:f:imperf	_	_	_	_	_
5	i	i	_	conj	_	_	_	_	_
6	z	z	_	prep:inst:nwok	_	_	_	_	_
7	fantazyą	fantazyą	_	subst:sg:inst:f	_	_	_	_	_
8	-	-	_	interp	_	_	_	_	_
9	dostawało	dostawać	_	praet:sg:n:imperf	_	_	_	_	_
10	się	się	_	part	_	_	_	_	_
11	po	po	_	prep:loc	_	_	_	_	_
12	łydkach	łydka	_	subst:pl:loc:f	_	_	_	_	_
13	batem	bat	_	subst:sg:inst:m3	_	_	_	_	_
14	–	–	_	interp	_	_	_	_	_
15	i	i	_	conj	_	_	_	_	_
16	nieraz	nieraz	_	adv	_	_	_	_	_
17	łzy	łza	_	subst:pl:nom:f	_	_	_	_	_
18	zakręciły	zakręcić	_	praet:pl:f:perf	_	_	_	_	_
19	się	się	_	part	_	_	_	_	_
20	w	w	_	prep:loc:nwok	_	_	_	_	_
21	oczach	oko	_	subst:pl:loc:n:col	_	_	_	_	_
22	–	–	_	interp	_	_	_	_	_
23	ale	ale	_	conj	_	_	_	_	_
24	honor	honor	_	subst:sg:acc:m3	_	_	_	_	_
25	koński	koński	_	adj:sg:acc:m3:pos	_	_	_	_	_
26	niepozwalał	niepozwalał	_	imps:perf	_	_	_	_	_
27	płakać	płakać	_	inf:imperf	_	_	_	_	_
28	–	–	_	interp	_	_	_	

1	Niezapomnę	Niezapomnę	_	subst:sg:nom:m1	_	_	_	_	_
2	tego	ten	_	adj:sg:gen:m3:pos	_	_	_	_	_
3	wyrazu	wyraz	_	subst:sg:gen:m3	_	_	_	_	_
4	twarzy	twarz	_	subst:sg:gen:f	_	_	_	_	_
5	Matki	matka	_	subst:sg:gen:f	_	_	_	_	_
6	jak	jak	_	adv:pos	_	_	_	_	_
7	ze	z	_	part:wok	_	_	_	_	_
8	złożonemi	złożonemi	_	prep:inst:nwok	_	_	_	_	_
9	rękoma	ręka	_	subst:pl:inst:f	_	_	_	_	_
10	oczy	oko	_	subst:pl:nom:n:col	_	_	_	_	_
11	wzniosła	wzniosły	_	adj:sg:nom:f:pos	_	_	_	_	_
12	ku	ku	_	prep:dat	_	_	_	_	_
13	Niebu	Niebo	_	subst:sg:dat:n:ncol	_	_	_	_	_
14	a	a	_	conj	_	_	_	_	_
15	Staś	Staś	_	subst:sg:dat:f	_	_	_	_	_
16	zakrwawiony	zakrwawić	_	ppas:sg:nom:m3:perf:aff	_	_	_	_	_
17	zrywa	zrywać	_	fin:sg:ter:imperf	_	_	_	_	_
18	się	się	_	part	_	_	_	_	_
19	prędko	prędko	_	adv:pos	_	_	_	_	_
20	i	i	_	conj	_	_	_	_	_
21	biegnie	biec	_	fin:sg:ter:imperf	_	_	_	_	_
22	wołając	wołać	_	pcon:imperf	_	_	_	_	_
23	żeby	żeby	_	comp	_	_	_	_	_
24	go	on	_	ppron3:sg:gen:m1:ter:nakc:npraep	_	_	_	_	_
25	niezostawili	niezostawili	_	

1	W	w	_	prep:loc:nwok	_	_	_	_	_
2	roku	rok	_	subst:sg:loc:m3	_	_	_	_	_
3	1830	1830	_	dig	_	_	_	_	_
4	przyszedł	przyjść	_	praet:sg:m1:perf	_	_	_	_	_
5	naymłodzy	naymłodzy	_	adj:sg:nom:m1:pos	_	_	_	_	_
6	brat	brat	_	subst:sg:nom:m1	_	_	_	_	_
7	móy	móy	_	adj:sg:nom:m1:pos	_	_	_	_	_
8	Ludwik	Ludwik	_	subst:sg:nom:m1	_	_	_	_	_
9	–	–	_	interp	_	_	_	_	_
10	pamiętam	pamiętać	_	fin:sg:pri:imperf	_	_	_	_	_
11	że	że	_	comp	_	_	_	_	_
12	tego	ten	_	adj:sg:gen:m3:pos	_	_	_	_	_
13	dnia	dzień	_	subst:sg:gen:m3	_	_	_	_	_
14	śni	śnić	_	fin:sg:ter:imperf	_	_	_	_	_
15	mi	ja	_	ppron12:sg:dat:f:pri:nakc	_	_	_	_	_
16	się	się	_	part	_	_	_	_	_
17	,	,	_	interp	_	_	_	_	_
18	że	że	_	comp	_	_	_	_	_
19	mię	ja	_	ppron12:sg:acc:m1:pri:nakc	_	_	_	_	_
20	ktoś	ktoś	_	subst:sg:nom:m1	_	_	_	_	_
21	ś	być	_	aglt:sg:sec:imperf:nwok	_	_	_	_	_
22	pyta	pytać	_	fin:sg:ter:imperf	_	_	_	_	_
23	co	co	_	comp	_	_	_	_	_
24	chcę	chcieć	_	fin:sg:pri:imperf	_	_	_	_	_
25	–	–	_	interp	_	_	_	_	_
26	braciszka	braciszek	_	subst:sg:acc:m1	_	_	

1	Ale	ale	_	conj	_	_	_	_	_
2	były	być	_	praet:pl:m3:imperf	_	_	_	_	_
3	to	to	_	pred	_	_	_	_	_
4	czasy	czas	_	subst:pl:nom:m3	_	_	_	_	_
5	zepsucia	zepsuć	_	ger:sg:gen:n:perf:aff	_	_	_	_	_
6	–	–	_	interp	_	_	_	_	_
7	rozwody	rozwód	_	subst:pl:nom:m3	_	_	_	_	_
8	separacje	separacja	_	subst:pl:nom:f	_	_	_	_	_
9	częste	częsty	_	adj:pl:nom:f:pos	_	_	_	_	_
10	–	–	_	interp	_	_	_	_	_
11	to	to	_	subst:sg:nom:n:ncol	_	_	_	_	_
12	też	też	_	part	_	_	_	_	_
13	i	i	_	conj	_	_	_	_	_
14	pożycie	pożyć	_	ger:sg:acc:n:perf:aff	_	_	_	_	_
15	moich	mój	_	adj:pl:gen:n:pos	_	_	_	_	_
16	Wujostwa	wujostwo	_	subst:pl:acc:n:ncol	_	_	_	_	_
17	niebyło	Niebyła	_	subst:sg:voc:f	_	_	_	_	_
18	nayszczęśliwsze	nayszczęśliwsze	_	subst:sg:voc:f	_	_	_	_	_
19	–	–	_	interp	_	_	_	_	_
20	Lubo	Luba	_	subst:sg:voc:m1	_	_	_	_	_
21	niebyło	Niebyła	_	subst:sg:voc:f	_	_	_	_	_
22	zdaje	zdawać	_	fin:sg:ter:imperf	_	_	_	_	_
23	mi	ja	_	ppron12:sg:dat:m1:pri:nakc	_	_	_	_	_
24	się	się	_	part	_	_	_	_	_
25	sądowey	sądowey	_	fin:sg:ter:imperf	_	

In [91]:
original_text = OriginalAnnotations(file, lowercase=False, original=original_file, nested=False)
print(original_text.sentences)

['Dziad mój Melchior urodzony roku 1741 we wsi Godziszewo parafii Rypnin syn komornika ziemskiego Jana Komornika ziemi Dobrrzyńskiej (a nie Antoniego – Antoni był starszym bratem ś.p. Melchiora – i nie rozumiem skąd się wzięła ta myłka, która później wdokumentach się powtarza) i matki Ewy z Pinińskich właścicieli Dóbr Strużewo z adlinencjami Puszczanki, Dąbrowy części, Ronantowizna, Żółtowizna, Będowszczyazna, Kmińszczyzna, Bęklowizna, Ruszkowizna.', 'A że śp Dziad mój był najmłodszy i piąty z pomiędzy Rodzeństwa – odstąpił swoją część najstarszemu bratu swojemu Mateuszowi komornikowi ziemi Dobrzyńskiej za ośm tysięcy tynfów.', 'Byli jeszcze bracia Wawrzyniec, Szymon oraz siostra Panna Urszula Czermińska.', 'Stryjów miał śp. Melchior czterech i jedną ciotkę – Antoniego Wawrzyńca ożenionego z Głuską z niej trzech synów Jakuba, Łukasza i Floyrana.', 'Maciej z Chądzyńską, Floryan z Korytyńską, Wawrzyniec z Muczynowską – mieli syna Ludwika – Ciotka za Rzotoławskim – wszystkich braci stryje

### EXECUTION

In [58]:
dag = morfeusz.analyse(u'W Szczebrzeszynie chrząszcz brzmi w trzcinie.')
dag_disamb = concraft.disamb(dag)

In [59]:
best_interpretation(dag_disamb)

[(0, 1, ('W', 'w', 'prep:loc:nwok', [], []), '1.0000', None, 'disamb'),
 (1,
  2,
  ('Szczebrzeszynie',
   'Szczebrzeszyn',
   'subst:sg:loc:m3',
   ['nazwa_geograficzna'],
   []),
  '1.0000',
  None,
  'disamb'),
 (2,
  3,
  ('chrząszcz', 'chrząszcz', 'subst:sg:nom:m2', ['nazwa_pospolita'], []),
  '1.0000',
  None,
  'disamb'),
 (3,
  4,
  ('brzmi', 'brzmieć:Vi~ię', 'fin:sg:ter:imperf', [], []),
  '1.0000',
  None,
  'disamb'),
 (4, 5, ('w', 'w', 'prep:loc:nwok', [], []), '1.0000', None, 'disamb'),
 (5,
  6,
  ('trzcinie', 'trzcina', 'subst:sg:loc:f', ['nazwa_pospolita'], []),
  '1.0000',
  None,
  'disamb'),
 (6, 7, ('.', '.', 'interp', [], []), '1.0000', 'eos', 'disamb')]